## Transform Refunds Data
1. Extract specific portion of the string from refund_reason using split function
1. Extract specific portion of the string from refund_reason using regexp_extract function
1. Extract date and time from the refund_timestamp
1. Write transformed data to the Silver schema in hive metastore [Default Location: /user/hive/warehouse]

In [0]:

df_refunds = spark.read.table('gizmobox.bronze.refunds')
display(df_refunds)

In [0]:
# 1.

from pyspark.sql import functions as F

df_extracted = (
    df_refunds
    .select(
        'refund_id',
        'payment_id',
        'refund_timestamp',
        'refund_amount',
        F.split('refund_reason', ':')[0].alias('refund_reason'),
        F.split('refund_reason', ':')[1].alias('refund_source')
    )
)
display(df_extracted)

In [0]:
# 2.

df_extracted_regex = (
    df_refunds
    .select(
        'refund_id',
        'payment_id',
        'refund_timestamp',
        'refund_amount',
        F.regexp_extract("refund_reason", "^([^:]+):", 1).alias("refund_reason"),
        F.regexp_extract("refund_reason", "^[^:]+:(.*)$", 1).alias("refund_source")
    )
)
display(df_extracted_regex)

In [0]:
# 3.

df_extracted_time = (
    df_refunds
    .select(
        'refund_id',
        'payment_id',
        F.date_format('refund_timestamp', 'yyyy-MM-dd').cast('date').alias('refund_date'),
        F.date_format('refund_timestamp', 'HH:mm:ss').alias('refund_time'),
        'refund_amount',
        F.regexp_extract("refund_reason", "^([^:]+):", 1).alias("refund_reason"),
        F.regexp_extract("refund_reason", "^[^:]+:(.*)$", 1).alias("refund_source")
    )
)
display(df_extracted_time)

In [0]:
# 4.
df_extracted_time.writeTo('gizmobox.silver.py_refunds').createOrReplace()